In [2]:
import torch

## TorchのTensorになれる

In [3]:
x = torch.empty(5, 3)
print(x)

tensor([[1.5444e+25, 1.6217e-19, 7.0062e+22],
        [1.6795e+08, 4.7423e+30, 4.7393e+30],
        [9.5461e-01, 4.4377e+27, 1.7975e+19],
        [4.6894e+27, 7.9463e+08, 3.2604e-12],
        [2.6209e+20, 4.1641e+12, 1.9434e-19]])


In [7]:
x = torch.rand(5, 3)
print(x)

tensor([[0.0152, 0.5176, 0.5384],
        [0.4654, 0.9400, 0.6017],
        [0.4510, 0.9721, 0.2547],
        [0.4383, 0.5300, 0.1403],
        [0.4684, 0.5713, 0.5103]])


In [9]:
y = torch.zeros(5, 3, dtype=torch.double)
print(y)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], dtype=torch.float64)


In [10]:
y + x

tensor([[0.0152, 0.5176, 0.5384],
        [0.4654, 0.9400, 0.6017],
        [0.4510, 0.9721, 0.2547],
        [0.4383, 0.5300, 0.1403],
        [0.4684, 0.5713, 0.5103]], dtype=torch.float64)

In [12]:
x = torch.tensor([5.5, 3])
print(x)
x = x.new_ones(5, 3, dtype=torch.double)      # 5×3の1を作成,引数の指定がない場合,dtype,deviceを引き継ぐ
print(x)
x = torch.randn_like(x, dtype=torch.float)    # override dtype!
print(x)                        

tensor([5.5000, 3.0000])
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
tensor([[ 1.0309,  1.9087, -0.0948],
        [ 0.8796,  0.0381, -1.1298],
        [-1.8677,  0.3382, -1.3528],
        [-0.6197,  0.4077,  0.4624],
        [-0.2204, -0.0828,  0.2701]])


###  足し算/掛け算

In [15]:
y = torch.rand(5, 3)
print(x + y)

result = torch.empty(5, 3)
torch.add(x, y, out=result)
print(result)
x * 7

tensor([[ 1.1963,  2.1394,  0.2226],
        [ 1.0594,  0.6911, -1.0956],
        [-1.4598,  0.8658, -0.4986],
        [-0.3411,  0.5242,  0.5180],
        [ 0.1674,  0.8446,  0.6387]])
tensor([[ 1.1963,  2.1394,  0.2226],
        [ 1.0594,  0.6911, -1.0956],
        [-1.4598,  0.8658, -0.4986],
        [-0.3411,  0.5242,  0.5180],
        [ 0.1674,  0.8446,  0.6387]])


tensor([[  7.2165,  13.3606,  -0.6634],
        [  6.1575,   0.2664,  -7.9085],
        [-13.0741,   2.3673,  -9.4695],
        [ -4.3379,   2.8541,   3.2369],
        [ -1.5428,  -0.5794,   1.8909]])

## 自動微分

In [59]:
X1 = torch.tensor(3. , requires_grad=True)
X2 = torch.tensor(2., requires_grad=True)

Y1 = 3*X1 + 2*X2
Y2 = 4*X1 + X2

In [62]:
print(torch.autograd.grad(Y1, [X1, X2], retain_graph=True))
print(torch.autograd.grad(Y2, [X1, X2], retain_graph=True))

(tensor(3.), tensor(2.))
(tensor(4.), tensor(1.))


## 演習1

In [77]:
print(torch.cuda.is_available())

False


## 演習2

In [68]:
print(torch.autograd.grad(torch.exp(X1), X1))
print(torch.autograd.grad(torch.cos(X1), X1))
print(torch.autograd.grad(torch.sin(X1), X1))
print(torch.autograd.grad(X1 ** 2, X1))
print(torch.autograd.grad(torch.exp(3 *X1)/(torch.exp(3*X1) + torch.exp(2*X1)), X1))

(tensor(20.0855),)
(tensor(-0.1411),)
(tensor(-0.9900),)
(tensor(6.),)
(tensor(0.0452),)


## 演習3

In [75]:
y = torch.exp(3 *X1)/(torch.exp(3*X1) + torch.exp(2*X1))
z1 = torch.exp(y)
z2 = torch.cos(y)
z3 = torch.sin(y)
z4 = y * y
z5 = torch.exp(3 *y)/(torch.exp(3*y) + torch.exp(2*y))

In [76]:
print(torch.autograd.grad(z1, X1,retain_graph=True))
print(torch.autograd.grad(z2, X1,retain_graph=True))
print(torch.autograd.grad(z3, X1,retain_graph=True))
print(torch.autograd.grad(z4, X1,retain_graph=True))
print(torch.autograd.grad(z5, X1,retain_graph=True))

(tensor(0.1171),)
(tensor(-0.0368),)
(tensor(0.0262),)
(tensor(0.0861),)
(tensor(0.0091),)
